Connect Robot, Thymio Suite must be open and thymio must be connected to pc!

In [3]:
import math
import time
import ast
import openai
import tdmclient.notebook
import tdmclient.tools
await tdmclient.notebook.start()

Thymio Program

In [5]:
%%run_python

state = 0

@onevent
def forward(distance):
    global motor_left_target, motor_right_target, timer_period
    timer_period[0] = 100 * distance
    motor_left_target = 200
    motor_right_target = 200
    emit("motion", 1)


@onevent
def backward(distance):
    global motor_left_target, motor_right_target, timer_period
    timer_period[0] = 100 * distance
    motor_left_target = -200
    motor_right_target = -200
    emit("motion", 1)



@onevent
def rotateleft(angle):
    global motor_left_target, motor_right_target, timer_period
    timer_period[0] = 35 * angle
    motor_left_target = -50
    motor_right_target = 100
    emit("motion", 1)


@onevent
def rotateright(angle):
    global motor_left_target, motor_right_target, timer_period
    timer_period[0] = 35 * angle
    motor_left_target = 100
    motor_right_target = -50
    emit("motion", 1)

@onevent
def prox():
    global motor_left_target, motor_right_target, prox_horizontal, state
    if state == 0:
        for i in range(5):
            if prox_horizontal[i] > 2000:
                state = 1
        if state == 1:
                timer_period[0] = 0
                motor_left_target = 0
                motor_right_target = 0
                timer_period[1] = 500
                emit("problem", 0)


@onevent
def timer0():
    global motor_left_target, motor_right_target
    timer_period[0] = 0
    motor_left_target = 0
    motor_right_target = 0
    emit("done", 0)



In [8]:
class Done(Exception):
    pass

class Problem(Exception):
    pass

def parse_input(task_str):
    actions = ast.literal_eval(task_str)
    return actions

def pop_and_send(event_list) -> bool:
    if len(event_list) > 0:
        event = event_list.pop(0)
        send_event(event[0], event[1])
        return True
    else:
        return False

def execute_actions(actions): 
    event_list = actions.copy()

    def on_event_data(node, event_name):
        data = get_event_data(event_name)
        print(data)
        if event_name == "done":
            clear_event_data(event_name)
            success = pop_and_send(event_list)
            if not success:
                raise Done
        elif event_name == "problem":
            clear_event_data(event_name)
            raise Problem
        else:
            clear_event_data(event_name)

    pop_and_send(event_list)
    try:
        tdmclient.notebook.process_events(all_nodes=True, on_event_data=on_event_data)
    except Done:
        print("Robot task completed successfully")
        return "done", None
    except Problem:
        print("Robot encounted a problem")
        return "problem", event_list
    
quit = False
tasks = [parse_input(input("Please input a list of actions for the robot to perform"))]
while not quit:
    response, left_over_events = execute_actions(tasks.pop())
    match response:
        case "done":
            if len(tasks) == 0:
                quit = True
        case "problem":
            if left_over_events is not None:
                tasks.append(left_over_events)
            tasks.append(parse_input(input("Robot encoutered a problem (*Insert short description*).\nPlease provice extra guidance.")))
            

[[1]]
[[0]]
[[1]]
[[0]]
[[1]]
[[0]]
[[1]]
[[0]]
[[1]]
[[0]]
[[1]]
[[0]]
[[1]]
[[0]]
[[1]]
[[0]]
[[1]]
[[0]]
Robot task completed successfully


ChatGPT integration